# Data Leakage - Vazamento de informações 

In [8]:
import pandas as pd
# Read the data
# troca o yes por True e no por False
data = pd.read_csv('CSV/aer-credit-card-data/AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no']) 

print(data)

# Select target
y = data.card

# Select predictors
X = data.drop(['card'], axis=1)

print("Number of rows in the dataset:", X.shape[0])
X.head()



       card  reports       age  income     share  expenditure  owner  selfemp  \
0      True        0  37.66667  4.5200  0.033270   124.983300   True    False   
1      True        0  33.25000  2.4200  0.005217     9.854167  False    False   
2      True        0  33.66667  4.5000  0.004156    15.000000   True    False   
3      True        0  30.50000  2.5400  0.065214   137.869200  False    False   
4      True        0  32.16667  9.7867  0.067051   546.503300   True    False   
...     ...      ...       ...     ...       ...          ...    ...      ...   
1314   True        0  33.58333  4.5660  0.002146     7.333333   True    False   
1315  False        5  23.91667  3.1920  0.000376     0.000000  False    False   
1316   True        0  40.58333  4.6000  0.026513   101.298300   True    False   
1317   True        0  32.83333  3.7000  0.008999    26.996670  False     True   
1318   True        0  48.25000  3.7000  0.111619   344.157500   True    False   

      dependents  months  m

,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5




Como este é um conjunto de dados pequeno, usaremos a validação cruzada para garantir medidas precisas da qualidade do modelo.


In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

#Como não há pré-processamento, não precisamos de um pipeline (usado de qualquer maneira como prática recomendada!)
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy: %f" % cv_scores.mean())

Cross-validation accuracy: 0.981052


#### Resumo dos dados:


- card: 1 se o pedido de cartão de crédito for aceito, 0 se não
- age: Idade n anos mais duodécimos de ano
- income: Renda anual (dividida por 10.000)
- reports: Número de relatórios depreciativos importantes
- share: Proporção entre despesas mensais com cartão de crédito e renda anual
- expenditure: Gasto médio mensal com cartão de crédito
- owner: 1 se possui casa própria, 0 se aluga
- selfempl: 1 se for autônomo, 0 se não
- dependents: 1 + número de dependentes
- months: Meses morando no endereço atual
- majorcards: Número dos principais cartões de crédito mantidos
- active: Número de contas de crédito ativas


In [11]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Fraction of those who did not receive a card and had no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Fraction of those who received a card and had no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Fraction of those who did not receive a card and had no expenditures: 1.00
Fraction of those who received a card and had no expenditures: 0.02


In [12]:
# Eliminar preditores com vazamento do conjunto de dados
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Avaliar o modelo com preditores com vazamento removidos
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-val accuracy: %f" % cv_scores.mean())


Cross-val accuracy: 0.835471


## Exercise: Data Leakage

Ideias hipoteticas - Não tinha exercicios